In [ ]:
import pandas as pd
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor as RFR

In [ ]:
HV_ccg = []
with open('./path/to/HighlyVariableCellCycleGeneslist.txt',mode="r") as cycle:
    for i in cycle:
        HV_ccg.append(i[:-1])
Regulators = list({'MYC', 'SMAD2', 'TFDP1', 'TFDP2','EP300','CCNA2','CCNH','CDK1','HDAC1','HDAC2','MAD2L2','MDM2','PLK1','PTTG1','RAD21','RBX1','YWHAB'})
GRNs = {}
for c in range(1,16):
    ckey = "Cell"+str(c)
    GRN_df = pd.read_csv("./path/to/GRN_{}.csv".format(ckey),index_col=0)
    for g in HV_ccg:
        if g not in GRN_df.index:
            GRN_df.loc[g] = np.zeros(len(GRN_df.columns))
    GRN_df = GRN_df.loc[Regulators]
    for g in HV_ccg:
        if g not in GRN_df.columns:
            GRN_df[g] = np.zeros(len(GRN_df.index)) 
    #print(GRN_df.min().min())
    GRNs[ckey] = GRN_df/GRN_df.max().max()

In [ ]:
fucci = pd.read_csv("./path/to/FucciInt_LiveCell.csv",index_col=0)
fucci.columns=["Green","Red","Time","Cell"]
Ginis = pd.DataFrame()
for seed in range(0,30):
    trainX = []
    trainY = []
    for c in range(1,16):
        ckey = "Cell"+str(c)
        trainX.append(GRNs[ckey].values.flatten())
        Greens = fucci[fucci["Cell"]==ckey]["Green"].values
        Reds = fucci[fucci["Cell"]==ckey]["Red"].values
        trainY.append(np.hstack([Greens,Reds]))
    rg = RFR(n_jobs=-1, random_state=seed,n_estimators=310,max_depth=4)
    rg.fit(trainX, trainY)
    Gini = pd.DataFrame(rg.feature_importances_,index = [i+">"+j for i in GRNs["Cell1"].index for j in GRNs["Cell1"].columns],
                columns=["Gini_{}".format(seed)])
    Ginis = pd.concat([Ginis,Gini],axis=1)
else:
    Ginisum = Ginis.sum(axis=1)
    Ginisum = Ginisum.sort_values(ascending=False)
    GiniS = pd.DataFrame(Ginisum)
    GiniS.columns=["Gini"]

In [ ]:
FI = GiniS
from kneed import KneeLocator
kn = KneeLocator(x=range(len(FI["Gini"].values,)), y=FI["Gini"].values, S=2., curve="convex", direction="decreasing")
print(round(kn.knee, 3))
import matplotlib as mpl
font = {'family' : 'Arial','size': 7}
mpl.rc('font', **font)
fig = plt.figure(figsize=(1.2,1.2))
ax1 = fig.add_subplot(111)
ax1.set_axisbelow(True)
ax1.tick_params(labelbottom="on",bottom="on", labelleft="on",left="on", length=2,width=1, color=black)
#ax2.tick_params(labelbottom="on",bottom="on", labelleft="on",left="on", length=1, color=black)
for j in ["left","bottom"]:
    ax1.spines[j].set_color(black)
for j in ["top","right"]:
    ax1.spines[j].set_color("w")
    #ax1.spines[j].set_width(1)
plt.ylim(-0.05,0.65)
plt.yticks([0,0.2,0.4,0.6])
FI.plot(ax=ax1,legend=False,color="gray",linewidth=1)
#FeatureImp_df.iloc[:1300].plot(ax=ax)
FI.iloc[:kn.knee].plot(ax=ax1,legend=False,color=red,linewidth=2)
plt.xlabel("")
plt.ylabel("")
ax1.xaxis.grid(True, which='major', linestyle='--', color ="lightgray",linewidth=0.5)
ax1.yaxis.grid(True, which='major', linestyle='--', color ="lightgray",linewidth=0.5)
ax1.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
#ax1.scatter(x=corrs,y=dist_fucci,s=12.5,color=colors_15[-3],edgecolor=black,linewidth=0.5)
plt.show()
HighEdges = FI.iloc[:kn.knee].index
LowEdges = FI.iloc[kn.knee:].index

In [ ]:
import networkx as nx

G=nx.DiGraph ()
reg_l = []
rsp_l = []
for comb in HIGH:
    reg, rsp = comb.split(">")
    if (reg in L.index)&(rsp in L.index):
        reg_l.append(reg)
        rsp_l.append(rsp)
        G.add_edge(reg, rsp, weight = 1)
reg_l = list(set(reg_l))
rsp_l = list(set(rsp_l))
edges = G.edges()
#edge_weights = [np.log10(G[u][v]['weight']*100)/2 for u,v in edges]
pos = pos_L
#pos = nx.spring_layout(G)
fig, ax = plt.subplots(1, 1, figsize=(1.8*3, 1.8*3))
nx.draw_networkx_nodes(G, pos, node_color = "w", node_size = 90, alpha = 1,
                       linewidths=3, edgecolors= black, 
                       nodelist=rsp_l)
nx.draw_networkx_nodes(G, pos, node_color = red, node_size = 150, alpha = 1,
                       linewidths=3, edgecolors= black, 
                       nodelist=list(set(Regulators_TF)&set(reg_l)))
nx.draw_networkx_nodes(G, pos, node_color = blue, node_size = 150, alpha = 1,
                       linewidths=3, edgecolors= black, 
                       nodelist=list(set(Regulators_CoF)&set(reg_l)))
#nx.draw_networkx_labels(G, pos,font_size=11, font_color="k", font_family='Arial',ax=ax)

#edge_width = [FI.loc[u+">"+v,"Gini"] *2 for u,v in edges]
#edge_width = [1.2**fluc_cv_df.loc[u+">"+v,"cv"] for u,v in edges]
two = []
two_width = []
one = []
one_width = []
low = []
low_width = []
for u,v in edges:
    if fluc_var_df.loc[u+">"+v,"var"] > 2:
        two.append((u,v))#"#9E4D50"
        two_width.append(0.38**fluc_var_df.loc[u+">"+v,"var"])
    elif fluc_var_df.loc[u+">"+v,"var"] > 1:
        one.append((u,v))#"#CDA4A3"
        one_width.append(0.38**fluc_var_df.loc[u+">"+v,"var"])
    else:
        low.append((u,v))#"#DCDDE0"
        low_width.append(0.38**fluc_var_df.loc[u+">"+v,"var"])
        
HighEdges_list = [(u,v) for u,v in G.edges() if u+">"+v in HighEdges]
High_edge_width = [FI.loc[u+">"+v,"Gini"]*17.5 for u,v in edges if u+">"+v in HighEdges]
LowEdges_list = [(u,v) for u,v in G.edges() if u+">"+v in LowEdges]
Low_edge_width = [FI.loc[u+">"+v,"Gini"]*17.5 for u,v in edges if u+">"+v in LowEdges]

arrows = nx.draw_networkx_edges(G,pos,width=Low_edge_width, edge_color="#DCDDE0",arrowsize=8,edge_cmap=cmap_r,edgelist=LowEdges_list)
#for a, w in zip(arrows, Low_edge_width):
#    a.set_mutation_scale(w+5)
#    a.set_joinstyle('miter')
#    a.set_capstyle('butt')
arrows = nx.draw_networkx_edges(G,pos,width=High_edge_width, edge_color=red,arrowsize=8,edge_cmap=cmap_r,edgelist=HighEdges_list)
#for a, w in zip(arrows, Low_edge_width):
#    a.set_mutation_scale(w+5)
#    a.set_joinstyle('miter')
#    a.set_capstyle('butt')

plt.tight_layout()
plt.axis("off")
#plt.savefig('/home/torii/HeLa_LC_TimeSeries/Figures/GRN_Significant_Var_HighGini.png', transparent=False, dpi=350, bbox_inches="tight")
#nx.draw_networkx_edge_labels(G, pos,edge_labels)

In [ ]:
import networkx as nx

G=nx.DiGraph ()
reg_l = []
rsp_l = []
for comb in HighEdges:
    reg, rsp = comb.split(">")
    if reg == rsp:continue
    reg_l.append(reg)
    rsp_l.append(rsp)
    G.add_edge(reg, rsp, weight = 1)
reg_l = list(set(reg_l))
rsp_l = list(set(rsp_l))
import igraph as ig
edgelist=[list(i) for i in G.edges]
#add_edgelist=[list(i) for i in G3.edges]
#edgelist += add_edgelist
count = 0
count_dict = {}
count_dict_r = {}
for i in edgelist:
    if i[0] not in count_dict.keys():
        count_dict[i[0]] = count
        count_dict_r[count] = i[0]
        count+=1
    if i[1] not in count_dict.keys():
        count_dict[i[1]] = count
        count_dict_r[count] = i[1]
        count+=1
Num_edgelist = [[count_dict[i[0]], count_dict[i[1]]] for i in edgelist]
g = ig.Graph(edges = Num_edgelist)
g.vs["name"] = [count_dict_r[i] for i in range(len(count_dict_r.keys()))]
kamada_kawai_positions = g.layout_kamada_kawai()
layout = g.layout_fruchterman_reingold(seed=kamada_kawai_positions,niter = 100)
fig, ax = plt.subplots(figsize=(5,5))
ig.plot(g, target=ax,layout=layout,        
        vertex_size=5,
        vertex_color="indianred",
        edge_width=1,
        edge_color="gray",
       )
L=pd.DataFrame(layout._coords,index=g.vs["name"],columns=["x","y"])
pos_L = {i:[j["x"], j["y"]] for i,j in L.iterrows()}

In [ ]:
import networkx as nx

G=nx.DiGraph ()
reg_l = []
rsp_l = []
for comb in list(HighEdges)+list(LowEdges):
    reg, rsp = comb.split(">")
    if reg == rsp:continue
    if (reg in L.index)&(rsp in L.index):
        reg_l.append(reg)
        rsp_l.append(rsp)
        G.add_edge(reg, rsp, weight = 1)
reg_l = list(set(reg_l))
rsp_l = list(set(rsp_l))
edges = G.edges()
pos = pos_L
fig, ax = plt.subplots(1, 1, figsize=(8,8))#4.05
#nx.draw_networkx_labels(G, pos,font_size=12, font_color="k", font_family='Arial',ax=ax)
for nd in G.nodes:
    if nd in Regulators_TF:
        color = "#823438"
    elif nd in Regulators_CoF:
        color = "#415A77"
    else:
        color = black
    nx.draw_networkx_labels(G,pos,labels={nd:nd},font_color=color,font_size=11, font_family='Arial',ax=ax)
two = []
two_width = []
one = []
one_width = []
low = []
low_width = []
for u,v in edges:
    if u==v:continue
    if fluc_var_df.loc[u+">"+v,"var"] > 2:
        two.append((u,v))#"#9E4D50"
        two_width.append(1+5.8**fluc_var_df.loc[u+">"+v,"var"])
    elif fluc_var_df.loc[u+">"+v,"var"] > 1:
        one.append((u,v))#"#CDA4A3"
        one_width.append(1+5.8**fluc_var_df.loc[u+">"+v,"var"])
    else:
        low.append((u,v))#"#DCDDE0"
        low_width.append(1+5.8**fluc_var_df.loc[u+">"+v,"var"])
HighEdges_list = [(u,v) for u,v in G.edges() if u+">"+v in HighEdges]
High_edge_width = [FI.loc[u+">"+v,"Gini"]*70 for u,v in edges if u+">"+v in HighEdges]
LowEdges_list = [(u,v) for u,v in G.edges() if u+">"+v in LowEdges]
Low_edge_width = [FI.loc[u+">"+v,"Gini"]*10 for u,v in edges if u+">"+v in LowEdges]
arrows = nx.draw_networkx_edges(G,pos,width=1.25,#High_edge_width,
                                edge_color="lightgray",arrowsize=9,edgelist=HighEdges_list)
for a in arrows:
    a.set_joinstyle('miter')
    a.set_capstyle('butt')
plt.tight_layout()
plt.axis("off")
#nx.draw_networkx_edge_labels(G, pos,edge_labels)